In [152]:
class Node:
    def __init__(self, a,b,name):
        self.name = name
        self.a = a
        self.b = b

    def determine_backpropagation_order(self):
        a_order, b_order = [],[]

        if self.a:
            a_order = self.a.determine_backpropagation_order()
        
        if self.b:
            b_order = self.b.determine_backpropagation_order()

        combined_order = self.topologically_ordered_merge(a_order, b_order)

        return [self] + combined_order

    def topologically_ordered_merge(self, a_order, b_order):
        b_node_vs_index = {node:index for (index,node) in enumerate(b_order)}

        merged = []
        a_pointer, b_pointer = 0,0
        for a_index, a_node in enumerate(a_order):
            if a_node in b_node_vs_index:
                b_index = b_node_vs_index[a_node]

                merged.extend(a_order[a_pointer:a_index])
                merged.extend(b_order[b_pointer:b_index])
                merged.extend([a_node])

                a_pointer = a_index + 1
                b_pointer = b_index + 1

        merged.extend(a_order[a_pointer:])
        merged.extend(b_order[b_pointer:])

        return merged


a = Node(None, None, 'A')
b = Node(a, None, 'B')
c = Node(a, None, 'C')
d = Node(b,c, 'D')
e = Node(b, d, 'E')
f = Node(d, None, 'F')
g = Node(f, None, 'G')
h = Node(e, g, 'H')    

In [154]:
[node.name for node in h.determine_backpropagation_order()]

['H', 'E', 'G', 'F', 'D', 'B', 'C', 'A']

In [146]:
x = [1,2,3]
x[4:]

[]

In [80]:
import numpy as np
from src.tensor import Tensor 
from src.module import Module
from src.functional import Linear, ReLU, MSELoss
from tests.util import get_numerical_gradient

x = Tensor(np.random.randn(6, 1))
# x = Tensor([[5],[2]])
# y = Tensor(np.zeros((10,6,1)))

x.value

array([[-1.00296886],
       [-0.26703768],
       [-1.73112204],
       [-1.19364881],
       [-1.71991329],
       [ 1.81823182]])

In [129]:
class Softmax(Module):
    def forward(self, x):
        # z = x - x.max()
        numerator = x - 0
        # numerator = x - x.max()
        # return z
        # numerator = x

        # numerator = np.e ** z
        # numerator = np.e ** x
        denominator = numerator.sum(axis=-2)
        return numerator / denominator
        # return (numerator.swapaxes(0,1) / denominator).swapaxes(0,1)

y_hat = Softmax()(x) ** 2

x.zero_grad()
y_hat.backwards()

analytical_gradient = x.grad
numerical_gradient = get_numerical_gradient(
    lambda x: Softmax()(x) ** 2, 
    x, 
    x
)

c:\users\rens\repos\nn\nn\src\tensor.py:229: RuntimeWarning: invalid value encountered in log
  gradient = np.log(self.a.value) * self.a.value ** self.b.value * self.grad


In [130]:
numerical_gradient

array([[0.22269428],
       [0.3104045 ],
       [0.13591106],
       [0.19996854],
       [0.13724695],
       [0.55893243]])

In [131]:
analytical_gradient

array([[ 0.10315773],
       [ 0.27857809],
       [-0.07040863],
       [ 0.05770627],
       [-0.06773685],
       [ 0.77563371]])

In [139]:
point = y_hat.a.a

print(type(point))
print(point.shape, point.grad.shape)
print(point.grad)
print()
print(type(point.a))
if point.a: print(point.a.shape, point.a.grad.shape)
print(type(point.b))
if point.b: print(point.b.shape, point.b.grad.shape)

<class 'src.tensor.Add'>
(6, 1) (6, 1)
[[0.22269415]
 [0.31040433]
 [0.13591097]
 [0.19996842]
 [0.13724686]
 [0.55893214]]

<class 'src.tensor.Tensor'>
(6, 1) (6, 1)
<class 'src.tensor.Tensor'>
() ()


In [17]:
axis = 0
operand = Tensor(np.random.randn(1,2,3,4,1))

output = operand.sum(axis) ** 2
output.backwards()

analytical_gradient = operand.grad
numerical_gradient = get_numerical_gradient(
    lambda x: x.sum(axis) ** 2, 
    operand, 
    operand
)



c:\users\rens\repos\nn\nn\src\tensor.py:229: RuntimeWarning: invalid value encountered in log
  gradient = np.log(self.a.value) * self.a.value ** self.b.value * self.grad


In [18]:
point = output.a

print(type(point))
print(point.shape, point.grad.shape)
print()
print(type(point.a))
if point.a: print(point.a.shape, point.a.grad.shape)
print(type(point.b))
if point.b: print(point.b.shape, point.b.grad.shape)

<class 'src.tensor.Sum'>
(2, 3, 4, 1) (2, 3, 4, 1)

<class 'src.tensor.Tensor'>
(1, 2, 3, 4, 1) (1, 2, 3, 4, 1)
<class 'NoneType'>


In [20]:
gradient = np.expand_dims(
    point.grad, point.axis
).repeat(
    point.a.shape[point.axis], 
    point.axis
)

In [25]:
point.a.undo_broadcasting(gradient)

array([[[[[-0.27829102],
          [ 0.85762105],
          [ 0.79798314],
          [ 1.86664492]],

         [[-0.33168568],
          [ 1.7247919 ],
          [ 0.19928603],
          [-2.86502141]],

         [[-0.82787865],
          [-1.36991289],
          [-0.54332345],
          [-2.27695471]]],


        [[[-1.06046408],
          [-2.45910261],
          [ 1.07515675],
          [-1.45368118]],

         [[-0.75100475],
          [ 0.29703797],
          [-0.76098292],
          [ 3.42728596]],

         [[ 0.95384727],
          [-2.89373414],
          [ 2.08313651],
          [ 0.21220115]]]]])

In [19]:
zero_grad = np.zeros(param.shape)
grad = param.grad

In [25]:
zero_grad.shape

(6, 1)

In [2]:
point = output.b.a.a.b.a

print(point.shape, point.grad.shape)
print(type(point.a))
if point.a: print(point.a.shape, point.a.grad.shape)
print(type(point.b))
if point.b: print(point.b.shape, point.b.grad.shape)


(10, 12, 1) (10, 12, 1)
<class 'src.tensor.Tensor'>
(12, 24) (12, 24)
<class 'src.tensor.Tensor'>
(10, 24, 1) (10, 24, 1)


In [1]:
import pytest
import numpy as np
from src.tensor import Tensor
from src.module import Module
from src.functional import Linear, ReLU, Softmax, MSELoss
from tests.util import get_numerical_gradient

class MLP(Module):
    def __init__(self):
        self.fc1 = Linear(24, 12)
        self.relu = ReLU()
        self.fc2 = Linear(12,1)
        self.softmax = Softmax()

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.softmax(out)
        # out = out.sum()
        return out


network = MLP()
input_tensor = Tensor(np.random.randn(3,24,1))
criterion = MSELoss()
target = Tensor(np.zeros((3,1)))

output = network(input_tensor)
# loss = criterion(target, output)
# loss.backwards()

# parameters = network.parameters()

# for name, parameter in parameters.items():
#     numerical_gradient = get_numerical_gradient(network, input_tensor, parameter)
#     analytical_gradient = parameter.grad


#     print(
#         'Parameter: {}\n'.format(name),
#         'Numerical: {}\n'.format(numerical_gradient[0]),
#         'Analytical: {}\n'.format(analytical_gradient[1]),
#     )
#     # assert np.allclose(analytical_gradient, numerical_gradient, rtol=1e-4)


In [2]:
preds = Tensor(np.random.random(output.shape))
target = Tensor(np.ones(output.shape))

loss =criterion(target, preds)
loss.backwards()

ValueError: non-broadcastable output operand with shape () doesn't match the broadcast shape (3,1,1)

In [14]:
point = loss.a.a

print(point.shape)
print(type(point).__name__)
print('a: ' + type(point.a).__name__, point.a.grad)
print('b: ' + type(point.b).__name__, point.b.grad)

# point.a.value.T @ point.grad
# point.grad @ point.b.value.swapaxes(-2,-1)

(3, 1, 1)
Add
a: Tensor [[[0.]]

 [[0.]]

 [[0.]]]
b: Mul [[[0.]]

 [[0.]]

 [[0.]]]


(3, 1, 1)

In [11]:
point.b.value.shape

(10, 24, 1)

In [90]:
point = output.a

print(point.shape)
print(type(point).__name__)
print('a: ' + type(point.a).__name__)
print('b: ' + type(point.b).__name__)

# point.a.value.T @ point.grad
point.grad @ point.b.value.swapaxes(-2,-1)

(10, 1)
Dot
a: Tensor
b: Tensor


array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 

In [72]:
point.grad.shape

(10, 1)

In [73]:
point.b.value.T.shape

(1, 24)

In [68]:
point.grad.shape

(10, 10, 1)

In [69]:
point.b.value.T.shape

(1, 24, 10)

In [52]:
output.a.shape

(10, 10, 1)

In [74]:
output.a.a.grad.shape

(10, 24)

In [146]:
numerical_gradient

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 

In [125]:
output = network(input)
output.backwards()

In [126]:
np.isclose(network.fc1.weights.grad, numerical_gradient, atol=1e-1)

array([[False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True],
       [False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False],
       [False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False],
       [ True,  True,  True,  True,

In [117]:
(network.fc1.weights.grad - numerical_gradient)

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e